# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#506 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846693732411                   -0.70    4.0
  2   -7.852307815529       -2.25       -1.53    1.0
  3   -7.852646108286       -3.47       -2.52    3.0
  4   -7.852646678120       -6.24       -3.38    2.5
  5   -7.852646686279       -8.09       -4.73    1.5
  6   -7.852646686728       -9.35       -5.54    3.0
  7   -7.852646686730      -11.64       -6.43    1.8
  8   -7.852646686730      -13.21       -7.86    2.2
  9   -7.852646686730      -15.05       -8.11    3.2
 10   -7.852646686730   +    -Inf       -9.10    3.0
 11   -7.852646686730   +  -15.05       -9.87    2.0
 12   -7.852646686730   +    -Inf      -10.92    1.8
 13   -7.852646686730      -14.35      -10.09    1.0
 14   -7.852646686730   +  -14.35      -10.02    1.0
 15   -7.852646686730      -15.05       -9.31    1.8
 16   -7.852646686730   +    -Inf       -9.24    1.0
 17   -7.852646686730      -14.75       -9.58 

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846844323641                   -0.70           4.0
  2   -7.852529882986       -2.25       -1.62   0.80    5.0
  3   -7.852608859678       -4.10       -2.79   0.80    5.2
  4   -7.852646568845       -4.42       -3.58   0.80    7.5
  5   -7.852646683374       -6.94       -4.53   0.80    8.8
  6   -7.852646686681       -8.48       -5.07   0.80   10.5
  7   -7.852646686724      -10.36       -6.11   0.80   11.2
  8   -7.852646686730      -11.27       -7.13   0.80   13.5
  9   -7.852646686730      -13.32       -7.87   0.80   15.5
 10   -7.852646686730   +    -Inf       -8.92   0.80   16.8
 11   -7.852646686730      -14.75       -9.44   0.80   18.0
 12   -7.852646686730   +  -14.27      -10.20   0.80   19.2
 13   -7.852646686730      -14.45      -11.00   0.80   20.5
 14   -7.852646686730   +  -14.75      -12.15   0.80   22.5


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.408769e+01     3.657001e+00
 * time: 0.5289669036865234
     1     1.363122e+00     2.038183e+00
 * time: 0.721869945526123
     2    -1.918731e+00     2.015495e+00
 * time: 0.745297908782959
     3    -3.866269e+00     1.563298e+00
 * time: 0.779465913772583
     4    -5.378404e+00     1.402114e+00
 * time: 0.8133540153503418
     5    -6.950111e+00     7.138518e-01
 * time: 0.8473770618438721
     6    -7.134487e+00     9.593012e-01
 * time: 0.871042013168335
     7    -7.673539e+00     6.766240e-01
 * time: 0.8941810131072998
     8    -7.751894e+00     8.849611e-01
 * time: 0.9176340103149414
     9    -7.792265e+00     6.394824e-01
 * time: 0.9414119720458984
    10    -7.813598e+00     7.507137e-01
 * time: 0.9649200439453125
    11    -7.830818e+00     3.976014e-01
 * time: 0.9884860515594482
    12    -7.838885e+00     3.264645e-01
 * time: 1.0120069980621338
    13    -7.845220e+00     6.851408e-02
 * time: 1.03562998771667

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846790953053                   -0.70    3.8
  2   -7.852310191712       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686489                   -2.55
  2   -7.852646686730       -9.62       -5.63
  3   -7.852646686730   +    -Inf      -11.05
  4   -7.852646686730      -14.75      -16.07
|ρ_newton - ρ_scf|  = 4.1459305652621365e-13
|ρ_newton - ρ_scfv| = 3.0056958632316096e-13
|ρ_newton - ρ_dm|   = 1.0133338831226043e-9
